In [ ]:
import numpy as np
from scipy.optimize import linprog
import imageio

### Loading the data
$\text{All the data needed were saved in the same folder as the code, running the code elsewhere will prompt errors}$

In [ ]:
c_red_path = "C_red.npy" #path to the C.npy file (red)
y_red_path = "y_red.npy" #path to the y.npy file (red)
A_inv_red_path = "A_inv_red.npy" #path to the A_inv.py file (red)

c_green_path = "C_green.npy" #path to the C.npy file (green)
y_green_path = "y_green.npy" #path to the y.npy file (green)
A_inv_green_path = "A_inv_green.npy" #path to the A_inv.py file (green)

c_blue_path = "C_blue.npy" #path to the C.npy file (blue)
y_blue_path = "y_blue.npy" #path to the y.npy file (blue)
A_inv_blue_path = "A_inv_blue.npy" #path to the A_inv.py file (blue)

$\text{Code that does optimization}$

In [ ]:
def solve(A, b):
  numRows = A.shape[0] 
  numCols = A.shape[1]

  vF = np.ones([2*numCols, 1])

  mAeq = np.concatenate((A, -A), axis=1)
  vBeq = b

  vLowerBound = np.full([2 * numCols, 1], 0)
  vUpperBound = np.full([2 * numCols, 1], np.inf)
  Bounds = np.concatenate((vLowerBound, vUpperBound), axis=1)

  result = linprog(vF, A_eq=mAeq, b_eq=vBeq, bounds=Bounds)
  vUV = result.x
  s = vUV[0:numCols] - vUV[numCols:];

  return s

In [ ]:
A_red = np.load(c_red_path);
b_red = np.load(y_red_path);
s_red = solve(A_red, b_red);
np.save("s_red", s_red); #saves the sparse vector 



A_green = np.load(c_green_path);
b_green = np.load(y_green_path);
s_green = solve(A_green, b_green);
np.save("s_green", s_green); #saves the sparse vector 



A_blue = np.load(c_blue_path);
b_blue = np.load(y_blue_path);
s_blue = solve(A_blue, b_blue);
np.save("s_blue", s_blue); #saves the sparse vector 

$\text{Getting the vector} x \text{from the vector } s \text{ via } x=As $

In [ ]:
A_inv_red = np.load(A_inv_red_path)
A_inv_red = A_inv_red.astype('float64')
A_red = np.linalg.inv(A_inv_red) 
x_red = A_red.dot(s_red)

print(x_red)

A_inv_green = np.load(A_inv_green_path)
A_inv_green = A_inv_green.astype('float64')
A_green = np.linalg.inv(A_inv_green) 
x_green = A_green.dot(s_green)

print(x_green)


A_inv_blue = np.load(A_inv_blue_path)
A_inv_blue = A_inv_blue.astype('float64')
A_blue = np.linalg.inv(A_inv_blue) 
x_blue = A_blue.dot(s_blue)

print(x_blue)

$\text{Getting the Images back from the vector } x$

In [ ]:
y_red = x_red.reshape((100,100), order='F')
y_green = x_green.reshape((100,100), order='F')
y_blue = x_blue.reshape((100,100), order='F')

y = np.zeros((100,100,3))
y[:,:,0] = y_red
y[:,:,1] = y_green
y[:,:,2] = y_blue

imageio.imwrite('goose_color_reconstructed.jpg', y)